### Setup

In [2]:
#@markdown ## **Setup**
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout, jslink, IntText, IntSlider
import requests
#Creates a text document using the Raw github url. This text document will be used as a .py file for imports
def import_text(location, file):
  url = 'https://raw.githubusercontent.com/byuccl/digital_design_colab2/master/%s/%s' % (location, file)
  resp = requests.get(url)
  with open(file, 'wb') as f:
    f.write(resp.content)


import_text("Exercises/luts_and_mux/files/", "frq_luts_mux.py")
import_text("Exercises/luts_and_mux/files/", "mc_luts_mux.py")
import_text("Templates/", "mcTemplate.py")

from frq_luts_mux import *
from mc_luts_mux import *
from mcTemplate import * 
create_mc_dict(get_data())



# What is a MUX

A Mux, short for multiplexer, is a digital device that can be used to select between multiple inputs. This can be very useful when trying to implement complicated logic with many inputs. In the first example we have 2 options A or B. The select signal can be driven to select A when sel is low or to select B when sel is high. We can increase our select options by increasing the number of inputs we have. The number of inputs a mux can support can be calculated by solving 2^n where n is the number of select signals. In order to select the input you want simply count from the top to the bottom starting at 0 until you reach your signal. Then convert that number into binary and input that number through the select signals. For example, if I wanted to select C on the 4 input mux(2nd image) I would count 0,1,2 then convert that 2 to binary 10. That is the signal I need to pass to the select signal s1 = 1 s0 = 0 to get the output of C. The mapping of select signals to output goes as follows: A 00, B 01, C 10, D 11

<div style="display: flex;">
  <div style="flex: 1;">
    <img src="https://raw.githubusercontent.com/byuccl/digital_design_colab2/master/Exercises/luts_and_mux/media/2:1Mux.png"
      width="250" height="" style="display: block; margin: 0 auto" />
  </div>
  <div style="flex: 1;">
    <img src="https://raw.githubusercontent.com/byuccl/digital_design_colab2/master/Exercises/luts_and_mux/media/4:1Mux.png"
      width="200" height="" style="display: block; margin: 0 auto" />
  </div>
</div>



We can convert the mux into a truth table to see what the value of out is in every possible input scenario. To do this start by counting in binary down the input signals A B S. Then go back and fill in the output signal. Remember the select signal is simply deciding which signal to pass to the output. In this simple example this means that if s is 0 then output = A, if S is 1 output = B. We could continue this example by making a truth table for 4 inputs. However there would be 32 possible inputs due to the 4 input signals and 2 select signals, creating a table to large to be shown here. 

|A|B|S|O|
|-|-|-|-|
|0|0|0|0|
|0|0|1|0|
|0|1|0|0|
|0|1|1|1|
|1|0|0|1|
|1|0|1|0|
|1|1|0|1|
|1|1|1|1|


Mux's are often used in stacks. For example, lets say we have 2 8 bit signals we want to choose between. What we end up doing is using a single control signal that goes to all 8 mux's and feeding each of the individual bits into a different Mux. 


# Chaining Multiplexers 

Sometimes we may not have the right size of mux that we need. We can achieve different size by chaining MUX's together. For example say we only had 2:1 MUX's available. Is it possible to build a 4:1 mux? The answer is yes but at a cost. We will need not only 2 but 3 2:1 multiplexers to achieve this. In the picture below we connect the first 2 MUX's together using the same select signal. However these 2 MUX's have 2 outputs that must be then tied into a 3rd mux where the second select signal chooses the final output. This form of chaining can be used to create any size mux we need.

<p align="left">
<img src="https://raw.githubusercontent.com/byuccl/digital_design_colab2/master/Exercises/luts_and_mux/media/4:1-2:1.png"
width="300" height="" style="display: block; margin: 0 auto" />
</p>

# Building a Logic Gate with a Multiplexer

Another tool to add to our belt is the ability to build logic gates out of multiplexers. As we will soon find out, our F.P.G.A's implement a lot of logic through built in multiplexers. Before we can discuss why this is significant let's look at how it is done. In the first image we have an example of an implemented AND gate. We drive the signal at s0 to a constant low state. We then feed the 2 inputs through the select and S1. This may seem a little strange at first, however it is perfectly fine to feed an input through the select port. The truth table on the right demonstrates that we have created an AND gate using a 2:1 multiplexer. The 2nd image is a demonstration of building a NOR gate. Some gates require more than just 2 inputs to be implemented, for example an OR gate cannot be implemented with just a 2:1 multiplexer. 

<div style="display: flex;">
  <div style="flex: 1;">
    <img src="https://raw.githubusercontent.com/byuccl/digital_design_colab2/master/Exercises/luts_and_mux/media/AND_GATE_MUX.png"
      width="350" height="" style="display: block; margin: 0 auto" />
  </div>
  <div style="flex: 1;">
    <img src="https://raw.githubusercontent.com/byuccl/digital_design_colab2/master/Exercises/luts_and_mux/media/NOR_Gate_MUX.png"
      width="350" height="" style="display: block; margin: 0 auto" />
  </div>
</div>

# What is a LUT

LUT stands for Look up table. This is a fancy name for a multiplexer who's inputs are pre-determined and constant. For example, a LUT could hold 16 values, so 4 bits would be needed to select between these 16 values. This essentially allows us to set the initial values of all the inputs and then select between these signals later creating a small memory space. This can also be called Read Only Memory or ROM.

<p align="left">
<img src="https://raw.githubusercontent.com/byuccl/digital_design_colab2/master/Exercises/luts_and_mux/media/Lut.png"
width="400" height="" style="display: block; margin: 0 auto" />
</p>

# What makes these things so important?

These simple principles are what make and F.P.G.A (Field programmable logic array) possible. An F.P.G.A is essentially a lot of luts and muxes aligned in rows and columns forming a large grid. When you begin to code in an hdl language you are programming logical statements with memory. Constants can be stored in these luts and dynamic memory can be stored in flip flops. We can use muxes to implement all levels of logic. What this means is when we Synthesize and implement our design the CAD tool is taking our logic statements and implementing it through Muxes, Luts and flip flops. The way these are activated is through the bitstream that is generated. This stream simply holds a bunch of 1's and 0's that go through the F.P.G.A and turn elements either on or off. Giving us a way to program hardware in the field. 

# Practice Questions    

In [ ]:
#@markdown Q1
print_frq_grid(1)

In [ ]:
#@markdown Q2
print_mc_grid(1)

In [ ]:
#@markdown Q3
print_mc_grid(2)

In [ ]:
#@markdown Q4
print_frq_grid(4)

In [ ]:
#@markdown Q5
print_frq_grid(6)

Click [Here](https://colab.research.google.com/github/byuccl/digital_design_colab2/blob/master/Exercises/assign_operators/assign_operators.ipynb) to move on to the Assign Operators Lesson.